# Technical Explainer Comparison

Compares explanations from GPT-4o-mini and Llama 3.2 for technical questions.

In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

load_dotenv(override=True)

openrouter_key = os.getenv('OPENROUTER_API_KEY')
openrouter_url = os.getenv('OPENROUTER_BASE_URL', 'https://openrouter.ai/api/v1')
ollama_url = os.getenv('OLLAMA_BASE_URL', 'http://localhost:11434/v1')

gpt_client = OpenAI(base_url=openrouter_url, api_key=openrouter_key)
ollama_client = OpenAI(base_url=ollama_url, api_key='ollama')

MODEL_GPT = os.getenv('OPENROUTER_MODEL', 'openai/gpt-4o-mini')
MODEL_LLAMA = os.getenv('OLLAMA_MODEL', 'llama3.2')

In [ ]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
def explain_with_gpt(question, stream=True):
    response = gpt_client.chat.completions.create(
        model=MODEL_GPT,
        messages=[{"role": "user", "content": question}],
        stream=stream
    )
    
    if stream:
        result = ""
        handle = display(Markdown("## GPT-4o-mini Explanation\n"), display_id=True)
        for chunk in response:
            if chunk.choices[0].delta.content:
                result += chunk.choices[0].delta.content
                update_display(Markdown(f"## GPT-4o-mini Explanation\n\n{result}"), display_id=handle.display_id)
        return result
    else:
        result = response.choices[0].message.content
        display(Markdown(f"## GPT-4o-mini Explanation\n\n{result}"))
        return result

gpt_explanation = explain_with_gpt(question)

In [ ]:
def explain_with_llama(question):
    response = ollama_client.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[{"role": "user", "content": question}]
    )
    result = response.choices[0].message.content
    display(Markdown(f"## Llama 3.2 Explanation\n\n{result}"))
    return result

llama_explanation = explain_with_llama(question)

In [ ]:
def compare_explanations(gpt_explanation, llama_explanation):
    comparison_prompt = f"""Compare these two technical explanations and highlight key differences:

GPT-4o-mini:
{gpt_explanation}

Llama 3.2:
{llama_explanation}

Provide a concise comparison focusing on clarity, accuracy, and helpfulness."""
    
    response = gpt_client.chat.completions.create(
        model=MODEL_GPT,
        messages=[{"role": "user", "content": comparison_prompt}]
    )
    result = response.choices[0].message.content
    display(Markdown(f"## Comparison\n\n{result}"))
    return result

compare_explanations(gpt_explanation, llama_explanation)